In [ ]:
try:
    import dlt
except ImportError:
    class dlt:
        def table(comment, **options):
            def _(f): return f
            return _
        def read(table_name):
            return spark.read.table(table_name)

from pyspark.sql import SparkSession
from gold_properties import GoldProcessor
spark = SparkSession.builder.getOrCreate()
go = GoldProcessor(spark)

@dlt.table(
    comment="Aggregated property valuations in gold layer",
    table_properties={"quality": "gold"}
)
def go_property_valuations():
    fact_df = dlt.read("si_fact_transaction")
    dim_df = dlt.read("si_dim_properties")
    return go.aggregate_property_valuations(fact_df, dim_df)

@dlt.table(
    comment="Neighborhood trends in gold layer",
    table_properties={"quality": "gold"}
)
def go_neighborhood_trends():
    dim_df = dlt.read("si_dim_properties")
    return go.build_neighborhood_trends(dim_df)